<h1 align="left">
  <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://autonomousvision.github.io/py123d/_static/123D_logo_transparent_white.svg" width="500">
    <source media="(prefers-color-scheme: light)" srcset="https://autonomousvision.github.io/py123d/_static/123D_logo_transparent_black.svg" width="500">
    <img alt="Logo" src="https://autonomousvision.github.io/py123d/_static/123D_logo_transparent_black.svg" width="500">
  </picture>
  <h2 align="left">123D: Scene Tutorial</h1>
</h1>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from py123d.api import SceneAPI, SceneFilter
from py123d.api.scene.arrow.arrow_scene_builder import ArrowSceneBuilder
from py123d.common.multithreading.worker_parallel import SingleMachineParallelExecutor

## 1.1 Download Demo Logs

## 1.2 Create Scenes by filtering the datasets



The logs store continuous driving recordings. Scenes in 123D are sequences that are extracted from a log, e.g. given a predefined duration and history.

In the example below, we filter some scenes from all logs with 8 second duration and 8 seconds temporal distance (making the scenes non-overlapping). If `None` is passed to the duration, the scene will contain the complete log.

This `SceneFilter` object is passed to a `SceneBuilder` object to query `SceneAPI`'s from the dataset.





In [ ]:
from py123d.datatypes.sensors import PinholeCamera, PinholeCameraType

scene_filter = SceneFilter(
    split_names=None,
    log_names=None,
    scene_uuids=None,
    duration_s=7.0,
    history_s=0.0,
    timestamp_threshold_s=7.0,
    pinhole_camera_types=[PinholeCameraType.PCAM_F0],
    shuffle=True,
)
scene_builder = ArrowSceneBuilder()
worker = SingleMachineParallelExecutor()

# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)
print(f"Found {len(scenes)} scenes.")

In [ ]:
datasplits = []

for scene in scenes:
    datasplits.append(scene.log_metadata.split)


print(set(datasplits))

## 1.2 Inspecting the Scene

Let's inspect a random scenefrom our dataset.

A scene has several different metadata objects attached to it:

`SceneMetadata`: Information how the scene was extracted from the log. Each timestep in the log has universally unique identifier (UUID). The UUID of the initial timestep also serves as identifier for scene filtering.

In [ ]:
scene: SceneAPI = np.random.choice(scenes)
scene_metadata = scene.scene_metadata
print(scene_metadata)
print("\nInitial UUID:", scene_metadata.initial_uuid)
print("Number of iterations:", scene_metadata.number_of_iterations)
print("Number of history iterations:", scene_metadata.number_of_history_iterations)
print("Duration (s):", scene_metadata.duration_s)
print("Iteration duration (s):", scene_metadata.iteration_duration_s)

`LogMetadata`: Information of the log the scene was extracted from. This object also includes data about the map (if available), or static information of the ego vehicle, e.g. the included sensors and vehicle parameters

In [ ]:
log_metadata = scene.log_metadata
log_metadata

`MapMetadata`: If the map is available, this object includes information about the location, wether the map is 3D (`map_has_z`), of the the map is defined per log (`map_is_local`).

In [ ]:
map_metadata = scene.map_metadata
map_metadata

## 1.3 Retrieving data from the `SceneAPI`

Different datasets might provide different modalities. In general, you can load data using a `scene.get_modality_at_iteration(iteration=...)`

If a modality is not available, the return will be `None`. The `TimePoint` is the only modality that is strictly required to be available in a `Scene

Let's look at some examples:


### 1.3.1 `TimePoint`

Current time step in microseconds.

In [ ]:
from py123d.datatypes.time import TimePoint

iteration = 0
timepoint: TimePoint = scene.get_timepoint_at_iteration(iteration=iteration)
print(f"Time at iteration {iteration}:", timepoint)

### 1.3.2 `EgoStateSE3` 
State of the ego vehicle in 3D with location and orientation.

In [ ]:
from py123d.datatypes.vehicle_state import EgoStateSE3

if (ego_state := scene.get_ego_state_at_iteration(iteration=iteration)) is not None:
    ego_state: EgoStateSE3

    print("Vehicle parameters\t", ego_state.vehicle_parameters)

    # The ego vehicles coordinate system is defined by it's rear-axle / IMU location.
    print("Rear axle location:\t", ego_state.rear_axle_se3.point_3d)
    print("Rear axle orientation:\t", ego_state.rear_axle_se3.quaternion)

    # You can also use the center pose
    print("Center location:\t", ego_state.center_se3.point_3d)
    print("Center orientation:\t", ego_state.center_se3.quaternion)

### 1.3.3 `BoxDetectionWrapper`

Object that contains all bounding boxes in the current time step

In [ ]:
from py123d.datatypes.detections import BoxDetectionWrapper

if (box_detections := scene.get_box_detections_at_iteration(iteration=iteration)) is not None:
    box_detections: BoxDetectionWrapper

    print(f"Number of boxes:{len(box_detections)}")

    if len(box_detections) > 0:
        box_detection = box_detections[0]
        print("\nFirst box:")
        print("Dataset Label:\t", box_detection.metadata.label)
        print("Default Label:\t", box_detection.metadata.default_label)
        print("Parameters:\t", box_detection.bounding_box_se3)

### 1.3.4 `PinholeCamera`
Object containing the camera observation with a pinhole model.

In [ ]:
available_pinhole_types = scene.available_pinhole_camera_types
print("Available pinhole camera types:\t", available_pinhole_types)

if len(available_pinhole_types) > 0:
    camera_type = np.random.choice(available_pinhole_types)
else:
    camera_type = PinholeCameraType.PCAM_F0  # Front facing camera

# NOTE: If a camera is not available, the return will be None
if (pinhole_camera := scene.get_pinhole_camera_at_iteration(iteration=iteration, camera_type=camera_type)) is not None:
    pinhole_camera: PinholeCamera

    print(f"\nCamera type:\t{camera_type}")

    print(f"Image shape:\t{pinhole_camera.image.shape}")
    print(f"Intrinsics:\t{pinhole_camera.metadata.intrinsics}")
    print(f"Distortion:\t{pinhole_camera.metadata.distortion}")

    plt.imshow(pinhole_camera.image)
    plt.title(f"Camera Type: {camera_type}")
    plt.show()

### 1.3.5 `LiDAR`
Object containing a point cloud of a single laser scanner.

In [ ]:
from py123d.datatypes.sensors import LiDAR, LiDARType

available_lidar_types = scene.available_lidar_types
print("Available LiDAR types:\t", available_lidar_types)

if len(available_lidar_types) > 0:
    lidar_type = np.random.choice(available_lidar_types)
else:
    lidar_type = LiDARType.LIDAR_TOP  # Top mounted LiDAR

if (lidar := scene.get_lidar_at_iteration(iteration=iteration, lidar_type=lidar_type)) is not None:
    lidar: LiDAR

    print(f"\nLiDAR type:\t{lidar_type}")
    print(f"Shape (NxM):\t{lidar.point_cloud.shape}")
    print(f"Features (M):\t{[(enum.name, enum.value) for enum in lidar.metadata.lidar_index]}")

    xy = lidar.xy

    plt.scatter(xy[:, 0], xy[:, 1], s=0.1, alpha=0.25, c="black")
    plt.title(f"LiDAR Type: {lidar_type}")
    plt.xlabel("X-forward [m]")
    plt.ylabel("Y-left [m]")
    plt.axis("equal")

    range_limit = 100  # meters
    plt.xlim(-range_limit, range_limit)
    plt.ylim(-range_limit, range_limit)
    plt.show()

### 1.3.6 `MapAPI`

The `MapAPI` can get retrieved from a scene directly. If the map is available, we plot the map with our default plotting function.
For further information, you can visit the map or visualization tutorial.

In [ ]:
from py123d.api import MapAPI
from py123d.geometry import Point2D
from py123d.visualization.matplotlib.observation import add_default_map_on_ax
from py123d.visualization.matplotlib.utils import add_non_repeating_legend_to_ax


def simple_map_visualization(map_api: MapAPI, center_2d: Point2D, map_radius: float = 100.0):
    """Helper to plot the map using matplotlib.

    :param map_api: The MapAPI to visualize
    :param center_2d: The center point of the map visualization
    :param map_radius: The radius around the center point to visualize
    """

    fsize = 8
    _, ax = plt.subplots(figsize=(fsize, fsize))
    add_default_map_on_ax(ax, map_api=map_api, point_2d=center_2d, radius=map_radius)
    add_non_repeating_legend_to_ax(ax)
    ax.set_aspect("equal")
    ax.set_xlim(center_2d.x - map_radius, center_2d.x + map_radius)
    ax.set_ylim(center_2d.y - map_radius, center_2d.y + map_radius)
    plt.show()


if (map_api := scene.get_map_api()) is not None:
    map_api: MapAPI

    if (ego_state := scene.get_ego_state_at_iteration(iteration=iteration)) is not None:
        center_2d = ego_state.center_se3.point_2d
    else:
        center_2d = Point2D.from_array(np.array([0.0, 0.0]))

    print("\nMapAPI is available.")
    print("Map Metadata:", map_api.map_metadata)
    simple_map_visualization(map_api=map_api, center_2d=center_2d)

### 1.3.7 Others:

You can find further modalities in the documentation of [`SceneAPI`](todo)